In [4]:
from google.cloud import bigquery
import pandas as pd

In [5]:

time = '2025-06-24'
SQL_QUERY =  f"""
SELECT
  sso_id,additional_info,campaign_details,campaign_name,platform,event_time
FROM
  `htz-common.raw_data.requests` 
WHERE
  TIMESTAMP_TRUNC(event_time, DAY) >= TIMESTAMP('{time}')
  AND page_type = 'Chatbot'
  AND campaign_name in ('response answer','open question')
  -- AND sso_id IS NOT NULL
ORDER BY
  event_time DESC;
"""
client = bigquery.Client()
query_job = client.query(SQL_QUERY)

results = query_job.result() 
df = results.to_dataframe()
df.head(2)

/Users/roeishlezinger/Desktop/movie-recommendation-chatbot/.venv/lib/python3.12/site-packages/google/cloud/bigquery/table.py:1900: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,sso_id,additional_info,campaign_details,campaign_name,platform,event_time
0,<NA>,"[{'key': 'session_id', 'value': '3'}]",אפשר משו טוב?,open question,Desktop,2025-06-24 13:50:46.248000+00:00
1,<NA>,"[{'key': 'session_id', 'value': '2'}]",אפשר דוקו איראני?,open question,Desktop,2025-06-24 13:50:07.690000+00:00


In [6]:
# preprocess the data

df['parsed_data'] = df['additional_info'].apply(lambda x: {item['key']: item['value'] for item in x})
expanded_df = pd.json_normalize(df['parsed_data'].tolist())
expanded_df = expanded_df.drop(columns=['sso_id'])
df_cleaned = df.drop(columns=['additional_info', 'parsed_data'])
df = pd.concat([df_cleaned, expanded_df], axis=1)
df['id'] = df['sso_id'].astype(str) + '_' + df['session_id'].astype(str)

df.head(2)

,sso_id,campaign_details,campaign_name,platform,event_time,session_id,total_time,rag_speed,timestamp,input_tokens,...,article_ids_1,error,remaining_user_messages,troll_triggered,conversation_key,genres,regenerate,media_type,streaming_platforms,id
0,<NA>,אפשר משו טוב?,open question,Desktop,2025-06-24 13:50:46.248000+00:00,3,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>_3
1,<NA>,אפשר דוקו איראני?,open question,Desktop,2025-06-24 13:50:07.690000+00:00,2,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>_2


In [7]:
df[df['campaign_details'].fillna('').str.contains('closing_question')]

,sso_id,campaign_details,campaign_name,platform,event_time,session_id,total_time,rag_speed,timestamp,input_tokens,...,article_ids_1,error,remaining_user_messages,troll_triggered,conversation_key,genres,regenerate,media_type,streaming_platforms,id


In [8]:
df['id'].value_counts().sort_values(ascending=False).head(10)

id
389683771565_10    36
9037014398_61      16
3057469657_43      12
8010962861_116     10
9037014398_6        9
9037014398_9        9
9037014398_10       7
8010962861_109      5
9037014398_3        5
<NA>_1              4
Name: count, dtype: int64

In [12]:
def parse_conversation(ids:str) -> str:
    """
    Parse the conversation from the DataFrame based on the provided id.
    """
    conversation = df[df['id'] == ids]
    conversation = conversation.sort_values(by='event_time', ascending=True)
    if conversation.empty:
        return "No conversation found for this ID."
    
    conversation_text = []
    for _, row in conversation.iterrows():
        if row['campaign_name'] == 'open question':
            text = f"{row['event_time']} - USER: {row['campaign_details']}"
        elif row['campaign_name'] == 'response answer':
            text = f"{row['event_time']} - LLM: {row['campaign_details']}"
        conversation_text.append(text)
    
    return "\n".join(conversation_text)

txt = parse_conversation('389683771565_10')
print(txt)

2025-06-24 06:08:12.266000+00:00 - USER: תן לי סרט פסיכולוגיה דרמה חדש
2025-06-24 06:08:34.498000+00:00 - USER: כן
2025-06-24 06:08:53.771000+00:00 - USER: סרט הוליוודי לא ישראלי ומהחודש האחרון
2025-06-24 06:08:53.790000+00:00 - LLM: <strong>אורון שמיר</strong> ממליץ על הסרט <strong>נוכחות</strong>, מותחן אימה שמתואר גם כדרמה משפחתית העוסקת בהתמודדות עם טראומה. הוא אמנם לא מפחיד במיוחד, אבל <strong>שמיר</strong> מציין שהחוויה הקולנועית שלו ייחודית ומעוררת מחשבה.

עוד המלצה מגיעה מ<strong>פבלו אוטין</strong>, על הסרט <strong>חדר משלו</strong>. זהו סרט״דרמה״ שמתמקד בעולם של בית ספר תיכון ובמערכות יחסים בין מורים לתלמידים, ומתאר התמודדות עם מצוקה נפשית וכלכלית. <strong>אוטין</strong> מציין שהסרט עוסק בעומק הפסיכולוגי של הנפש האנושית.
2025-06-24 06:08:54.011000+00:00 - LLM: <strong>אורון שמיר</strong> ממליץ על הסרט <strong>נוכחות</strong>, מותחן אימה שמתואר גם כדרמה משפחתית העוסקת בהתמודדות עם טראומה. הוא אמנם לא מפחיד במיוחד, אבל <strong>שמיר</strong> מציין שהחוויה הקולנועית שלו ייחודית ו

In [10]:
from pydantic import BaseModel, Field

class ReviewChat(BaseModel):
    quality: int = Field(..., description="Quality rating of the chat conversation, from 1 to 5, where 1 is poor and 5 is excellent.")
    feedback: str = Field(..., description="Feedback on the chat conversation.")
    


In [13]:
from google import genai
import os

model_name = "gemini-2.5-pro"
system_instruction = """
You are an LLM as a judge designed to test the quality of the output of a RAG-based chatbot for movie and series recommendations. 
You will receive parameters for examining the conversation between the user and the language model, 
and you must return an answer as to whether the correspondence meets the criteria you received.
"""

client = genai.Client(api_key=os.environ.get("GOOGLE_API_KEY"))
temp = client.models.generate_content(
    model=model_name,
    contents=txt,
                config={
            'response_mime_type': 'application/json',
            'response_schema': ReviewChat,
            'system_instruction': system_instruction,
        },
)



In [14]:
temp.parsed

ReviewChat(quality=1, feedback="הצ'אט בוט לא הבין את הבקשות של המשתמש לאורך כל השיחה. המשתמש ביקש סרט הוליוודי, לא ישראלי, והצ'אט בוט הציע לו סרטים ישראליים. בנוסף, המשתמש ביקש המלצה לסרט מז'אנר מסוים, והצ'אט בוט הציע לו סרטים מז'אנרים שונים לחלוטין. הצ'אט בוט גם לא התייחס לבקשות של המשתמש לסרטים מפלטפורמות סטרימינג ספציפיות. בשלב מסוים המשתמש התייאש וביקש סרט המבוסס על סיפור אמיתי, אך הצ'אט בוט טען שהוא לא יכול לספק המלצות כאלה. התשובות של הצ'אט בוט היו לא רלוונטיות, והוא חזר על עצמו מספר פעמים, מה שהוביל לתסכול מצד המשתמש. בסוף השיחה הצ'אט בוט הציע המלצה לא קשורה בעליל, מה שמעיד על חוסר הבנה מוחלט של השיחה.")

In [ ]:
txt

'2025-06-24 06:08:12.266000+00:00 - USER: תן לי סרט פסיכולוגיה דרמה חדש\n2025-06-24 06:08:34.498000+00:00 - USER: כן\n2025-06-24 06:08:53.771000+00:00 - USER: סרט הוליוודי לא ישראלי ומהחודש האחרון\n2025-06-24 06:08:53.790000+00:00 - LLM: <strong>אורון שמיר</strong> ממליץ על הסרט <strong>נוכחות</strong>, מותחן אימה שמתואר גם כדרמה משפחתית העוסקת בהתמודדות עם טראומה. הוא אמנם לא מפחיד במיוחד, אבל <strong>שמיר</strong> מציין שהחוויה הקולנועית שלו ייחודית ומעוררת מחשבה.\n\nעוד המלצה מגיעה מ<strong>פבלו אוטין</strong>, על הסרט <strong>חדר משלו</strong>. זהו סרט״דרמה״ שמתמקד בעולם של בית ספר תיכון ובמערכות יחסים בין מורים לתלמידים, ומתאר התמודדות עם מצוקה נפשית וכלכלית. <strong>אוטין</strong> מציין שהסרט עוסק בעומק הפסיכולוגי של הנפש האנושית.\n2025-06-24 06:08:54.011000+00:00 - LLM: <strong>אורון שמיר</strong> ממליץ על הסרט <strong>נוכחות</strong>, מותחן אימה שמתואר גם כדרמה משפחתית העוסקת בהתמודדות עם טראומה. הוא אמנם לא מפחיד במיוחד, אבל <strong>שמיר</strong> מציין שהחוויה הקולנועית שלו יי